# Koneoppiminen: Tehtävä 6: Koneoppimismallien vertailu (max 15 p)

In [1]:
# Kirjoita tähän tietosi!
student_name = 'Opiskelijan nimi tai Opiskelijoiden nimet'
student_id = 'JAMK-opiskelijatunnus tai JAMK-opiskelijatunnukset'


Tehtävässä vertaillaan eri koneoppimismenetelmät (kuusi eri menetelmää) samalle datalle.
Tallennetaan tulokset ja analysoidaan ne.
Esitetään tuloksia myös visuaalisesti (Ennustetarkkuus, sekaannusmatriisi, ROC-käyrä).
Lisäksi vertaillaan saatuja tuloksia sekä pohditaan tuloksia sanallisesti.

## Aineisto: Äänet

Tiedostossa [voice.csv](./voice.csv) on mittaustietoja äänistä (kenttien selitykset tiedostossa ovat englanniksi).

Pyri ennustamaan kentän `label (male/female)` arvoa muiden kenttien avulla eri koneoppimismenetelmien avulla.

Jaa data opetusdataan ja testidataan suhteessa 80/20%.

* Tulosta myös kuinka paljon eri kentät vaikuttavat mallin mukaan ennustettavaan muuttujaan (*importances*).
* Valitse sitten kaksi tärkeintä selittävää muuttujaa ja tulosta näistä scatter-kaavio niin että *male/female* -pisteet ovat eri värillä.

## Tehtävä 6.1
### Datan esikäsittely (2 pistettä)

Tutustu aineistoon ja toteuta datan esikäsittely.
* Esikäsittelyn toteutuksessa kannattaa huomioida, että `male` ja `female` kenttien arvot oltava numeerisia koneoppimisvaiheessa

In [2]:
# TODO: Kirjoita toteutuksesi tähän soluun.
import pandas as pd

df = pd.DataFrame({1:['1']})  # TODO: Tämän voi poistaa, kun aloittaa tehtävän tekemisen

# TODO: Esikäsittelyn toteutus

df

,1
0,1


## Tehtävä 6.2
### Koulutus- ja testiaineisto (2 pistettä)

Tehdään datan jako opetus- ja testijoukkoon sekä skaalataan numeerinen data.

In [3]:
# TODO: Kirjoita toteutuksesi tähän soluun.

X_train = X_test = y_train = y_test = []

## Tehtävä 6.3
### Mallien rakentaminen (7 pistettä)

Ajetaan eri oppimismenetelmät samalle datalle ja tallennetaan tulokset.
Tässä tehtävässä käytetään seuraavia menetelmiä:
1. Päätöspuu (1p)
2. Tukivektorikone eli SVM (1p)
3. k lähintä naapuria eli kNN (1p)
4. Naiivi Nayes Bayes (1p)
5. Satunnaismetsä eli RandomForest (1p)
6. PCA - osa 6 esimerkin aihe (2 p)

Laske kaikissa menetelmissä luokittelutuloksien **tarkkuus** ja **sekaannusmatriisi**.


In [4]:
# TODO: Malli 1: Päätöspuun toteutus

In [5]:
# TODO: Malli 2: Tukivektorikoneen toteutus

In [6]:
# TODO: Malli 3: kNN toteutus

In [7]:
# TODO: Malli 4: Bayes toteutus

In [8]:
# TODO: Malli 5: Satunnaismetsän toteutus

In [9]:
# TODO: Malli 6: PCA yhdistettynä johonkin luokittelumalliin

## Tehtävä 6.4
### Tulosten analysointi ja käsittely (4 p)

* Verrataan eri menetelmistä saatuja tuloksia toisiinsa.
* Pyri myös hakemaan keskeiset piirteet, jotka positiivisesti vaikuttavat saatuihin tuloksiin. 
   * Piirrä jokin soveltuva kuvio kahden tärkeimmän selittävän muuttujan avulla.
* Piirrä myös ROC - käyrät eri menetelmiä käytettäessä ja vertaa, miten eri koneoppimismenetelmissä etenee..
* Sanallinen pohdinta: Miten kehittäisit tai parantaisit mallien saamia oppimistuloksia? Pohdi, mitkä mallit voisit toimia vielä paremmin kuin tehtävissä annetut luokittelumallit?

#### Vastaus: Tulosten tulkinta

* TODO: MENETELMIEN VERTAILUA VOI TEHDÄ SANALLISESTI SAATUJEN TULOSTEN JA KUVIEN AVULLA.
* TODO: TULOSTEN SANALLISTA TULKINTAA VASTATEN KYSYMYKSIIN, JOITA TEHTÄVÄSSÄ ESITETTY
* TODO: MUUTAKIN SAA TÄHÄN KOMMENTOIDA TEHTÄVÄÄN LIITTYEN.
